In [1]:
import syft as sy
import torch as th
import numpy as np
import copy
from sympy import symbols
import random
sy.VERBOSE = False

Skipping torch.torch.Tensor.__div__ not supported in 1.4.0
Skipping torch.torch.Tensor.__floordiv__ not supported in 1.4.0
Skipping torch.torch.Tensor.__rfloordiv__ not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_and not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_and_ not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_or not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_or_ not supported in 1.4.0
Skipping torch.torch.Tensor.div not supported in 1.4.0
Skipping torch.torch.Tensor.div_ not supported in 1.4.0
Skipping torch.torch.Tensor.floor_divide not supported in 1.4.0
Skipping torch.torch.Tensor.floor_divide_ not supported in 1.4.0
Skipping torch.torch.Tensor.is_meta not supported in 1.4.0
Skipping torch.torch.Tensor.logical_and not supported in 1.4.0
Skipping torch.torch.Tensor.logical_and_ not supported in 1.4.0
Skipping torch.torch.Tensor.logical_or not supported in 1.4.0
Skipping torch.torch.Tensor.logical_or_ not supported in 1.4.0
Skipping 

In [ ]:
duet = sy.launch_duet()
# duet = sy.launch_duet(budget_database='~./duet/privacy_budgets.db')
# duet.store

🎤  🎸  ♪♪♪ starting duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently 
♫♫♫ > in alpha. Do not use this to protect real-world data.
♫♫♫ >
♫♫♫ > Punching through firewall to OpenGrid Network Node at network_url: 
♫♫♫ > http://ec2-18-216-8-163.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... Skipping torch.torch.Tensor.__div__ not supported in 1.4.0
Skipping torch.torch.Tensor.__floordiv__ not supported in 1.4.0
Skipping torch.torch.Tensor.__rfloordiv__ not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_and not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_and_ not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_or not supported in 1.4.0
Skipping torch.torch.Tensor.bitwise_or_ not supported in 1.4.0
Skipping torch.torch.Tensor.div not supported in 1.4.0
Skipping torch.torch.Tensor.div_ not supported in 1.4.0
Skipping torch.torch.Tensor.floor_divide not supported in 1.4.0
Skipping torch.torch.

In [12]:
# Utility functions

def gen_privscal(value, entity, range_min=0., range_max=10.):
    # generate private scalar
    p_id = str(entity) + str(PrivateScalar.create_id())
    x = {entity:{p_id:value}}
    g = symbols(p_id)
    f = {entity:{p_id:range_min}} 
    c = {entity:{p_id:range_max}}
    return PrivateScalar(x, g, f, c, p_id)

def concat_nestdicts(a, b):
    # concatenate 2 nested dictionaries
    x = {}
    for ent, val_dict in a.items():
        x[ent] = copy.deepcopy(val_dict)
    for ent, val_dict  in b.items():
        if ent not in x:
            x[ent] = copy.deepcopy(val_dict)
        else:
            for val_id, val in val_dict.items():
                x[ent][val_id] = val
    return x

In [4]:
class PrivateScalar:
    
    def __init__(self, x, g, f, c, p_id):
        
        self.p_id = p_id
        self.x = x # intermediate data
        self.g = g # polynomial
        self.f = f # lower bound val
        self.c = c # upper bound class
        
    def __repr__(self):
        return str("<p_id: " + self.p_id 
                   + " x: " + str(self.x)
                   + " g: " + str(self.g)
                   + " f: " + str(self.f)
                   + " c: " + str(self.c)
                  + ">")
    
    @staticmethod
    def create_id():
        return str(random.randint(0, 1e10))
    
    def __add__(self, other):
        x = concat_nestdicts(self.x, other.x)
        f = concat_nestdicts(self.f, other.f)
        c = concat_nestdicts(self.c, other.c)
        g = self.g + other.g
        p_id = self.create_id()
        return PrivateScalar(x, g, f, c, p_id)
    
    def __mul__(self, other):
        x = concat_nestdicts(self.x, other.x)
        f = concat_nestdicts(self.f, other.f)
        c = concat_nestdicts(self.c, other.c)
        g = self.g * other.g
        p_id = self.create_id()
        return PrivateScalar(x, g, f, c, p_id)

In [20]:
# Example testing

x = [1.5, 2, 3, 4, 5]
entities = ['bob', 'kriti', 'andrew', 'bob', 'amber'] # every value corresponds to an entity
num_entities = len(entities) # ??? number of unique entities instead
priv_a = gen_privscal(x[0], entities[0])
priv_b = gen_privscal(x[2], entities[2])
print(priv_a * priv_b)
print(priv_a + priv_b)
print(priv_a)
print(priv_b)

<p_id: 405173733 x: {'bob': {'bob4140844245': 1.5}, 'andrew': {'andrew2620106328': 3}} g: andrew2620106328*bob4140844245>
<p_id: 5028770490 x: {'bob': {'bob4140844245': 1.5}, 'andrew': {'andrew2620106328': 3}} g: andrew2620106328 + bob4140844245>
<p_id: bob4140844245 x: {'bob': {'bob4140844245': 1.5}} g: bob4140844245>
<p_id: andrew2620106328 x: {'andrew': {'andrew2620106328': 3}} g: andrew2620106328>


In [9]:
class RDP_Accountant:
    
    def __init__(self, entities, num_unique_entities, ac_id, l2_norm, alpha, lipschitz, sigma, data_profile):
        self.act_id = ac_id
        self.data_profile = data_profile # every row, column or value corresponds to an entity
        self.entities = entities # dic
        self.num_unique_entities = num_unique_entities
        self.num_entities = len(entities)
        # num_entities >= num_unique_entities
        
        # below budget values are upper bounds (remaining at the start)
        self.data_budget = th.tensor([1.] * num_entities)
        self.global_data_budget = 10.0
        self.scientist_budget = th.tensor([0.1] * num_entities)
        self.global_scientist_budget = 2.0
        
        self.rdp_guarantee = th.tensor([0.] * num_entities)
        self.global_rdp_guarantee = 0.
        self.l2_norm = l2_norm # array of values per entity
        self.alpha = alpha # alpha > 1.0 
        self.lipschitz = lipschitz # array of values per entity
        self.sigma = sigma
        
        # current value of budget spent per entity 
        self.eps = (alpha * (l2_norm ** 2) * (lipschitz ** 2)) / (2 * (sigma ** 2)) 
        # array of values per entity
        
    def update_budgets(self):
        self.data_budget -= self.eps
        self.rdp_guarantee = self.eps
    
#     def public_approx_Renyi_filter(self, max_rounds):
#         # input: public approx list of chosen epsilons and deltas (per round), 
#         #        public global epsilon & delta, round of computation t / T individual filtering 
#         # output: Continue / Halt execution
#         return True
    
    def public_approx_Renyi_odometer(self, max_rounds):
        # gives a valid upper bound on the privacy loss incurred thus far
        # input: round of computation t / T
        
        # add upper and lower bounds on the below values of budget remaining
#         data_budget = th.tensor([1.] * num_entities)
#         global_data_budget = 10.0
#         scientist_budget = th.tensor([0.1] * num_entities)
#         global_scientist_budget = 2.0
        rdp_guarantee = tuple(alpha, eps)
        return rdp_guarantee
    
    def add_laplacian_noise(self, eps, l1_sens):
        return 1.0
    
    def add_gaussian_noise(self, eps, l2_sens, delta):
        return 1.0

In [26]:
class PrivateTensor:
    
    # each row is a data point, number of data points = no. of columns. 2-D tensor
    def __init__(self, tensor, min_tensor, max_tensor, entity_cont, num_entities):
        self.data = tensor
        self.entity_cont = entity_cont # entity contribution to each data point 
        # self.data[i] = sum over all j entity_cont[i][j]
        self.data_min = min_tensor
        self.data_max = max_tensor
        self.lipschitz = []
        self.l2_norm = []

x + 1

In [ ]:
# duet.budget

print("OVERALL - Per Scientist Entity Budget:" + str(3))
print("OVERALL - Universal Entity Budget:" + str(5))
print()
print("EST. REMAINING - MIN Universal Entity Budget:" + str(4.2))
print("EST. REMAINING - MAX Universal Entity Budget:" + str(4.9))
print("EST. REMAINING - MEAN Universal Entity Budget:" + str(4.21))
print()
print("EST. REMAINING - MIN Per Scientist Entity Budget:" + str(2.91))
print("EST. REMAINING - MAX Per Scientist Entity Budget:" + str(3.0))
print("EST. REMAINING - MEAN Per Scientist Entity Budget:" + str(2.92))

# estimated budget spend for this tensor
print("EST SPEND - MIN Per Scientist Entity Budget:" + str(0))
print("EST SPEND - MAX Per Scientist Entity Budget:" + str(0))
print("EST SPEND - MEAN Per Scientist Entity Budget:" + str(0.12))

# show analytics again as percentage ?

In [ ]:
#Figuring stuff out
x = [1, 2, 3, 4]
y = [2, 4, 6, 7]
ent_x = [bob, andrew, amber, bob]
y = sum(x) + x[0]
y = 2 * bob[0] + andrew[0] + amber[0] # this is wrong

# from sympy import symbols
x, y = symbols('x y')
print(x)
print(y)
print(y * (x * 2))

expr = x + y
# expr = expr.subs(x, 0)
# expr = expr.subs(y, 1)
expr.evalf(subs={x: 2.4, y:0})
# expr
# print(expr)

# ages = th.tensor([23,52,31,16]).tag("ages").private(every_value_unique_entity=True, # every_row_unique_entity=False,
#                                             # every_column_unique_entity=False,
#                                             max_value=125, min_value=0).send(duet, searchable=True)
# # ages.entities by default equals = [acc.n_entities+0,acc.n_entities+1,acc.n_entities+2,acc.n_entities+3]
# heights = th.tensor([150,160,140,100]).tag("heights").private(every_value_unique_entity=True, max_value=250, 
#                                             min_value=0, entities=ages.entities).send(duet, searchable=True)